# Day1: Sentiment analysis with hotel reviews - processing the data

In [3]:
import pandas as pd
import time

# importing time so the start and end time can be used to calculate file loading time
print("Loading data file now, this could take a while depending on file size")
start = time.time()

# df is 'DataFrame' - make sure you downloaded the file to the data folder
df = pd.read_csv('../data/Hotel_Reviews.csv')
end = time.time()

print("Loading took " + str(round(end - start, 2)) + " seconds")

Loading data file now, this could take a while depending on file size
Loading took 6.29 seconds


In [4]:
print("The shape is the number of rows and columns is ",df.shape)

The shape is the number of rows and columns (515738, 17)


In [24]:
## Calculate the frequency count for reviewer nationalities:

# How many distinct values are there for the column Reviewer_Nationality and what are they?
national_freq = df['Reviewer_Nationality'].value_counts()
print("There are "+str(national_freq.size)+" distinct values of the column Reviewer_Nationality")


There are 227 distinct values of the column Reviewer_Nationality
Reviewer_Nationality
 United Kingdom               245246
 United States of America      35437
 Australia                     21686
 Ireland                       14827
 United Arab Emirates          10235
                               ...  
 Cape Verde                        1
 Northern Mariana Islands          1
 Tuvalu                            1
 Guinea                            1
 Palau                             1
Name: count, Length: 227, dtype: int64



In [26]:
# What reviewer nationality is the most common in the dataset (print country and number of reviews)?
print(df['Reviewer_Nationality'].value_counts())


Reviewer_Nationality
 United Kingdom               245246
 United States of America      35437
 Australia                     21686
 Ireland                       14827
 United Arab Emirates          10235
                               ...  
 Cape Verde                        1
 Northern Mariana Islands          1
 Tuvalu                            1
 Guinea                            1
 Palau                             1
Name: count, Length: 227, dtype: int64


In [35]:
# What are the next top 10 most frequently found nationalities, and their frequency count?\
print("The most frequently reviewed nation of the top 10 most reviewer nationalities is",national_freq[0])
print("\nThe next top 10 most frequently found nationalities, and their frequency count:  \n",national_freq[1:11])

The most frequently reviewed nation of the top 10 most reviewer nationalities is 245246

The next top 10 most frequently found nationalities, and their frequency count:  
 Reviewer_Nationality
 United States of America     35437
 Australia                    21686
 Ireland                      14827
 United Arab Emirates         10235
 Saudi Arabia                  8951
 Netherlands                   8772
 Switzerland                   8678
 Germany                       7941
 Canada                        7894
 France                        7296
Name: count, dtype: int64


In [57]:
# What was the most frequently reviewed hotel for each of the top 10 most reviewer nationalities?
for nation in national_freq[:10].index:
    nation_df = df[df["Reviewer_Nationality"]== nation]
    nation_freq = nation_df["Hotel_Name"].value_counts()
    print("The most frequently reviewed hotel for", nation.strip(), "is", nation_freq.index[0], "with", nation_freq[0],"reviews")

The most frequently reviewed hotel for United Kingdom is Britannia International Hotel Canary Wharf with 3833 reviews
The most frequently reviewed hotel for United States of America is Hotel Esther a with 423 reviews
The most frequently reviewed hotel for Australia is Park Plaza Westminster Bridge London with 167 reviews
The most frequently reviewed hotel for Ireland is Copthorne Tara Hotel London Kensington with 239 reviews
The most frequently reviewed hotel for United Arab Emirates is Millennium Hotel London Knightsbridge with 129 reviews
The most frequently reviewed hotel for Saudi Arabia is The Cumberland A Guoman Hotel with 142 reviews
The most frequently reviewed hotel for Netherlands is Jaz Amsterdam with 97 reviews
The most frequently reviewed hotel for Switzerland is Hotel Da Vinci with 97 reviews
The most frequently reviewed hotel for Germany is Hotel Da Vinci with 86 reviews
The most frequently reviewed hotel for Canada is St James Court A Taj Hotel London with 61 reviews


In [73]:
# First create a new dataframe based on the old one, removing the uneeded columns
hotel_freq_df = df.drop(["Hotel_Address", "Additional_Number_of_Scoring", "Review_Date", "Average_Score", "Reviewer_Nationality", "Negative_Review", "Review_Total_Negative_Word_Counts", "Positive_Review", "Review_Total_Positive_Word_Counts", "Total_Number_of_Reviews_Reviewer_Has_Given", "Reviewer_Score", "Tags", "days_since_review", "lat", "lng"], axis = 1)

# Group the rows by Hotel_Name, count them and put the result in a new column Total_Reviews_Found
hotel_freq_df['Total_Reviews_Found'] = hotel_freq_df.groupby('Hotel_Name').transform('count')

# Get rid of all the duplicated rows
hotel_freq_df = hotel_freq_df.drop_duplicates(subset = ["Hotel_Name"])
display(hotel_freq_df) 

,Hotel_Name,Total_Number_of_Reviews,Total_Reviews_Found
0,Hotel Arena,1403,405
405,K K Hotel George,1831,566
971,Apex Temple Court Hotel,2619,1037
2008,The Park Grand London Paddington,4380,1770
3778,Monhotel Lounge SPA,171,35
...,...,...,...
511962,Suite Hotel 900 m zur Oper,3461,439
512401,Hotel Amadeus,717,144
512545,The Berkeley,232,100
512645,Holiday Inn London Kensington,5945,2768


In [74]:
# define a function that takes a row and performs some calculation with it
def get_difference_review_avg(row):
  return row["Average_Score"] - row["Calc_Average_Score"]

# 'mean' is mathematical word for 'average'
df['Calc_Average_Score'] = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)

# Add a new column with the difference between the two average scores
df["Average_Score_Difference"] = df.apply(get_difference_review_avg, axis = 1)

# Create a df without all the duplicates of Hotel_Name (so only 1 row per hotel)
review_scores_df = df.drop_duplicates(subset = ["Hotel_Name"])

# Sort the dataframe to find the lowest and highest average score difference
review_scores_df = review_scores_df.sort_values(by=["Average_Score_Difference"])

display(review_scores_df[["Average_Score_Difference", "Average_Score", "Calc_Average_Score", "Hotel_Name"]])

,Average_Score_Difference,Average_Score,Calc_Average_Score,Hotel_Name
495945,-0.8,7.7,8.5,Best Western Hotel Astoria
111027,-0.7,8.8,9.5,Hotel Stendhal Place Vend me Paris MGallery by...
43688,-0.7,7.5,8.2,Mercure Paris Porte d Orleans
178253,-0.7,7.9,8.6,Renaissance Paris Vendome Hotel
218258,-0.5,7.0,7.5,Hotel Royal Elys es
...,...,...,...,...
151416,0.7,7.8,7.1,Best Western Allegro Nation
22189,0.8,7.1,6.3,Holiday Inn Paris Montparnasse Pasteur
250308,0.9,8.6,7.7,MARQUIS Faubourg St Honor Relais Ch teaux
68936,0.9,6.8,5.9,Villa Eugenie


In [76]:
# without lambdas (using a mixture of notations to show you can use both)
start = time.time()
no_negative_reviews = sum(df.Negative_Review == "No Negative")
print("Number of No Negative reviews: " + str(no_negative_reviews))

no_positive_reviews = sum(df["Positive_Review"] == "No Positive")
print("Number of No Positive reviews: " + str(no_positive_reviews))

both_no_reviews = sum((df.Negative_Review == "No Negative") & (df.Positive_Review == "No Positive"))
print("Number of both No Negative and No Positive reviews: " + str(both_no_reviews))

end = time.time()
print("Sum took " + str(round(end - start, 2)) + " seconds")



Number of No Negative reviews: 127890
Number of No Positive reviews: 35946
Number of both No Negative and No Positive reviews: 127
Sum took 0.55 seconds
